# Capstone Project - The Battle of Neighborhoods


## Business Problem section
Today construction companies need to choose carefully the places where they plan to build, because today people have a lot of options to choose, so it is important that a construction company attracts their clients by giving them a strategic location, so what if we can create a model to choose the better location for a living building, I will do it for a medium size city like Medellin, Colombia. It is needed that you know that Medellin is divided in 16Th districts, so I will recompile dates from Foursquare API to determine which is the best district to construct base in types and amenities closed to each one.


## Data section
District location will be obtained by manual google search, and to explore across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization

## Metodology


## Import required Packages

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

## insert Medellin districst location

In [23]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_af745c8d732b4e0daef28900624f62cb = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='AxEm8ympZgvR3v54DggTS8ytN2aD09V1AgnmGLMhyJaS',
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_af745c8d732b4e0daef28900624f62cb.get_object(Bucket='finalproject-donotdelete-pr-srnssg7dxiq64f',Key='Medellin districts2.xlsx')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df = pd.read_excel(body)
df.head()


,Distric Name,Latitude,Longitude
0,Popular,6.303598,-75.546573
1,Santa Cruz,6.293128,-75.558847
2,Manrique,6.267475,-75.567122
3,Aranjuez,6.283251,-75.567719
4,Castilla,6.292563,-75.575080


In [24]:
# create map of Medellin using latitude and longitude values
latitude=6.239309
longitude=-75.579580
map_medellin= folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, distric in zip(df['Latitude'], df['Longitude'], df['Distric Name']):
    label = '{}'.format(df)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_medellin)  
    
map_medellin

In [25]:

CLIENT_ID = '4MCPJ2DIEEYUW50ZTA5ZU3UHS2TUWPPXZK0C152GXMDIK01W'
CLIENT_SECRET = 'VSXWHH3RLINL2SINJCJQ03WYUFWWKPMFQ0MLFZUYHFJJS4BD'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 4MCPJ2DIEEYUW50ZTA5ZU3UHS2TUWPPXZK0C152GXMDIK01W
CLIENT_SECRET:VSXWHH3RLINL2SINJCJQ03WYUFWWKPMFQ0MLFZUYHFJJS4BD


In [37]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Distric Name', 
                  'Distric Latitude', 
                  'Distric Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
medellin_venues = getNearbyVenues(names=df['Distric Name'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Popular
Santa Cruz
Manrique
Aranjuez
Castilla
Doce de octubre
Robledo
Villa Hermosa
Buenos Aires
La Candelaria
Laureles—Estadio
La América
San Javier
Poblado
Guayabal
Belén


## consolidate data

In [44]:
print(medellin_venues.shape)
medellin_venues.head()

(592, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Popular,6.303598,-75.546573,Periódico Punto De Encuentro,6.304401,-75.548536,Business Service
1,Popular,6.303598,-75.546573,Aserrio Madepátula,6.310807,-75.550608,Home Service
2,Popular,6.303598,-75.546573,Metrocable Linea K - Estación Popular,6.295100,-75.548188,Cable Car
3,Popular,6.303598,-75.546573,Panne Cook Gourmet Express,6.294913,-75.544692,Sandwich Place
4,Santa Cruz,6.293128,-75.558847,METRO - Estacion Tricentenario,6.290542,-75.564733,Metro Station


## count of values per Distric

In [46]:
medellin_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Aranjuez,15,15,15,15,15,15
Belén,70,70,70,70,70,70
Buenos Aires,15,15,15,15,15,15
Castilla,6,6,6,6,6,6
Doce de octubre,5,5,5,5,5,5
Guayabal,32,32,32,32,32,32
La América,75,75,75,75,75,75
La Candelaria,92,92,92,92,92,92
Laureles—Estadio,100,100,100,100,100,100


## here we star data transformation to futter analasis

In [48]:
print('There are {} uniques categories.'.format(len(medellin_venues['Venue Category'].unique())))

There are 146 uniques categories.


In [49]:
medellin_onehot = pd.get_dummies(medellin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
medellin_onehot['Neighborhood'] = medellin_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [medellin_onehot.columns[-1]] + list(medellin_onehot.columns[:-1])
medellin_onehot = medellin_onehot[fixed_columns]

medellin_onehot.head()

,Neighborhood,Airport,Airport Lounge,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Cable Car,Café,Caribbean Restaurant,Cemetery,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gastropub,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Home Service,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Market,Metro Station,Mexican Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Outdoor Event Space,Outdoor Supply Store,Paella Restaurant,Park,Pastry Shop,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Pie Shop,Pizza Place,Planetarium,Plaza,Pool,Pub,Recreation Center,Rental Service,Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Stadium,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Tour Provider,Toy / Game Store,Track Stadium,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint
0,Popular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Popular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Popular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Popular,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Santa Cruz,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [50]:
medellin_onehot.shape

(592, 147)

In [51]:
medellin_grouped = medellin_onehot.groupby('Neighborhood').mean().reset_index()
medellin_grouped

,Neighborhood,Airport,Airport Lounge,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Auditorium,Auto Garage,Auto Workshop,BBQ Joint,Bakery,Bar,Basketball Court,Bed & Breakfast,Beer Garden,Big Box Store,Bistro,Boarding House,Botanical Garden,Boutique,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Bus Station,Business Service,Cable Car,Café,Caribbean Restaurant,Cemetery,Clothing Store,Cocktail Bar,Coffee Shop,Colombian Restaurant,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Department Store,Dessert Shop,Diner,Doner Restaurant,Donut Shop,Electronics Store,Factory,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden Center,Gastropub,General Entertainment,Grocery Store,Gym,Gym / Fitness Center,Health Food Store,Historic Site,Home Service,Hostel,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Juice Bar,Korean Restaurant,Latin American Restaurant,Liquor Store,Market,Metro Station,Mexican Restaurant,Mountain,Movie Theater,Multiplex,Museum,Music Store,Nature Preserve,Outdoor Event Space,Outdoor Supply Store,Paella Restaurant,Park,Pastry Shop,Pedestrian Plaza,Peruvian Restaurant,Pet Store,Pie Shop,Pizza Place,Planetarium,Plaza,Pool,Pub,Recreation Center,Rental Service,Restaurant,Road,Rock Climbing Spot,Salad Place,Salon / Barbershop,Salsa Club,Sandwich Place,Scenic Lookout,School,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Stadium,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Theater,Theme Park,Tour Provider,Toy / Game Store,Track Stadium,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Wings Joint
0,Aranjuez,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.066667,0.00,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.000000,0.00,0.066667,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.066667,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.133333,0.00,0.000000,0.066667,0.000000,0.000000,0.00000,0.00,0.000000,0.000000,0.00000,0.066667,0.0,0.066667,0.00000,0.00000,0.00000,0.066667,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.066667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00,0.00000,0.000000
1,Belén,0.00000,0.00000,0.000000,0.042857,0.000000,0.000000,0.000000,0.00,0.014286,0.00,0.000000,0.00000,0.000000,0.014286,0.100000,0.00,0.00000,0.014286,0.00000,0.00,0.00,0.000000,0.000000,0.00,0.028571,0.057143,0.000000,0.000000,0.00,0.000000,0.042857,0.000000,0.000000,0.000000,0.014286,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.014286,0.000000,0.014286,0.014286,0.000000,0.014286,0.000000,0.000000,0.014286,0.000000,0.00,0.000000,0.014286,0.000000,0.014286,0.028571,0.00,0.014286,0.000000,0.00,0.00,0.000000,0.000000,0.014286,0.028571,0.014286,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.028571,0.00,0.00000,0.042857,0.014286,0.00000,0.000000,0.000000,0.00,0.014286,0.000000,0.014286,0.0,0.00000,0.01

In [53]:
medellin_grouped.shape

(16, 147)

In [55]:
num_top_venues = 5

for hood in medellin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = medellin_grouped[medellin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aranjuez----
                 venue  freq
0          Bus Station  0.13
1                Plaza  0.13
2    Recreation Center  0.07
3        Metro Station  0.07
4  Fried Chicken Joint  0.07


----Belén----
              venue  freq
0               Bar  0.10
1       Pizza Place  0.07
2      Burger Joint  0.06
3    Sandwich Place  0.04
4  Arepa Restaurant  0.04


----Buenos Aires----
                        venue  freq
0         Fried Chicken Joint  0.07
1                 Supermarket  0.07
2                    Boutique  0.07
3              Ice Cream Shop  0.07
4  Construction & Landscaping  0.07


----Castilla----
                        venue  freq
0  Construction & Landscaping  0.17
1                Home Service  0.17
2                Tennis Court  0.17
3                         Gym  0.17
4                      School  0.17


----Doce de octubre----
               venue  freq
0  Convenience Store   0.2
1  Recreation Center   0.2
2     Scenic Lookout   0.2
3           Mountain   0.2
4 

In [56]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [57]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = medellin_grouped['Neighborhood']

for ind in np.arange(medellin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(medellin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aranjuez,Plaza,Bus Station,Sandwich Place,Fried Chicken Joint,Donut Shop,Metro Station,Storage Facility,Fast Food Restaurant,School,Shopping Mall
1,Belén,Bar,Pizza Place,Burger Joint,Sandwich Place,Arepa Restaurant,Italian Restaurant,Café,Vegetarian / Vegan Restaurant,Breakfast Spot,Shopping Mall
2,Buenos Aires,Shopping Mall,Bar,Burrito Place,Supermarket,Department Store,Fried Chicken Joint,Park,Nature Preserve,Plaza,Ice Cream Shop
3,Castilla,Home Service,Gym,School,Construction & Landscaping,Tennis Court,Recreation Center,Concert Hall,Comfort Food Restaurant,Convenience Store,Cosmetics Shop
4,Doce de octubre,Recreation Center,Mountain,Convenience Store,Scenic Lookout,Theater,Department Store,Electronics Store,Donut Shop,Doner Restaurant,Diner


In [60]:
# set number of clusters
kclusters = 5

medellin_grouped_clustering = medellin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(medellin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 4, 0, 0, 0, 0, 0], dtype=int32)

## cloustering

In [65]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

medellin_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
medellin_merged = medellin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Distric Name')

medellin_merged.head() # check the last columns!

,Distric Name,Latitude,Longitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Popular,6.303598,-75.546573,2,2,Home Service,Sandwich Place,Cable Car,Business Service,Convenience Store,Cosmetics Shop,Creperie,Construction & Landscaping,Concert Hall,Colombian Restaurant
1,Santa Cruz,6.293128,-75.558847,1,1,Metro Station,Cable Car,Fast Food Restaurant,Factory,Department Store,Electronics Store,Donut Shop,Doner Restaurant,Diner,Dessert Shop
2,Manrique,6.267475,-75.567122,0,0,Café,Theme Park,Shopping Mall,Supermarket,Latin American Restaurant,Sandwich Place,Botanical Garden,Concert Hall,Multiplex,Museum
3,Aranjuez,6.283251,-75.567719,0,0,Plaza,Bus Station,Sandwich Place,Fried Chicken Joint,Donut Shop,Metro Station,Storage Facility,Fast Food Restaurant,School,Shopping Mall
4,Castilla,6.292563,-75.575080,3,3,Home Service,Gym,School,Construction & Landscaping,Tennis Court,Recreation Center,Concert Hall,Comfort Food Restaurant,Convenience Store,Cosmetics Shop


In [67]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(medellin_merged['Latitude'], medellin_merged['Longitude'], medellin_merged['Distric Name'], medellin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [69]:
medellin_merged.loc[medellin_merged['Cluster Label'] == 0, medellin_merged.columns[[1] + list(range(5, medellin_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,6.267475,Café,Theme Park,Shopping Mall,Supermarket,Latin American Restaurant,Sandwich Place,Botanical Garden,Concert Hall,Multiplex,Museum
3,6.283251,Plaza,Bus Station,Sandwich Place,Fried Chicken Joint,Donut Shop,Metro Station,Storage Facility,Fast Food Restaurant,School,Shopping Mall
6,6.277659,Gym / Fitness Center,Fried Chicken Joint,Hotel,Music Store,Grocery Store,Rental Service,Shopping Mall,Park,Cupcake Shop,Diner
7,6.248326,Plaza,Pizza Place,Bar,Farmers Market,Park,Boutique,Burrito Place,Food Court,Latin American Restaurant,Steakhouse
8,6.237032,Shopping Mall,Bar,Burrito Place,Supermarket,Department Store,Fried Chicken Joint,Park,Nature Preserve,Plaza,Ice Cream Shop
9,6.250067,Plaza,Sandwich Place,Restaurant,Café,Bar,Department Store,Clothing Store,Shopping Mall,Fast Food Restaurant,Theater
10,6.249579,Burger Joint,Café,Pizza Place,Italian Restaurant,Restaurant,Bar,Sandwich Place,Athletics & Sports,BBQ Joint,Bakery
11,6.251677,Café,Pizza Place,Burger Joint,Mexican Restaurant,Coffee Shop,Gym,Park,Restaurant,Sushi Restaurant,Supermarket
12,6.255162,Ice Cream Shop,Park,Art Gallery,General Entertainment,Café,Metro Station,Snack Place,Street Art,Wings Joint,Electronics Store
13,6.208658,Café,Burger Joint,Hotel,Restaurant,Bar,Pizza Place,Cocktail Bar,Breakfast Spot,Argentinian Restaurant,Colombian Restaurant


In [70]:
medellin_merged.loc[medellin_merged['Cluster Label'] == 1, medellin_merged.columns[[1] + list(range(5, medellin_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,6.293128,Metro Station,Cable Car,Fast Food Restaurant,Factory,Department Store,Electronics Store,Donut Shop,Doner Restaurant,Diner,Dessert Shop


In [71]:
medellin_merged.loc[medellin_merged['Cluster Label'] == 2, medellin_merged.columns[[1] + list(range(5, medellin_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,6.303598,Home Service,Sandwich Place,Cable Car,Business Service,Convenience Store,Cosmetics Shop,Creperie,Construction & Landscaping,Concert Hall,Colombian Restaurant


In [72]:
medellin_merged.loc[medellin_merged['Cluster Label'] == 3, medellin_merged.columns[[1] + list(range(5, medellin_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,6.292563,Home Service,Gym,School,Construction & Landscaping,Tennis Court,Recreation Center,Concert Hall,Comfort Food Restaurant,Convenience Store,Cosmetics Shop


In [74]:
medellin_merged.loc[medellin_merged['Cluster Label'] == 4, medellin_merged.columns[[1] + list(range(5, medellin_merged.shape[1]))]]

,Latitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,6.303261,Recreation Center,Mountain,Convenience Store,Scenic Lookout,Theater,Department Store,Electronics Store,Donut Shop,Doner Restaurant,Diner


## Results and discussion

After all the analysis we obtain that we have 2 Districts that are huge better than the others, these are: Poblado and Laureles, they have a huge concentrations of revenue places, so it will be ideal places to construction companies to build.

other important conclusion is that as a construction company we have to be far away from districts located in cluster 1 to 5, these locations are totally far to the revenue places of the city.

I can conclude that do this analysis in a medium size city implies low data, and some of it should be obtained manually.

It will be great to compare other medium size cities in order to look better places of construction


## conclusion 

Data analysis can be a powerful tool that can help construction sector, improving sales because they can know the best place to construct but without doing a expensive marketing analysis, just acquiring and analyzing the right data
